# Raw Data Processing

This is to reformat and pre-process the original data.

## Imports and Constants

In [1]:
import re
import os
import csv
import numpy as np
import pandas as pd
import multiprocessing as mp

In [2]:
RAW_DATASET_DIR = 'Raw datasets/'
FMCG_DATASET_FILEPATH = RAW_DATASET_DIR + 'fmcg.csv'

PROCESSED_OUTPUT_DIR = 'Processed Datasets/'
PROCESSED_DATASET_REFORMATTED_DIR = PROCESSED_OUTPUT_DIR + 'reformatted/'

COMMODITY_MAPPING  = {
    "01 - FOOD AND NON-ALCOHOLIC BEVERAGES": "Food_and_drinks",
    "02 - ALCOHOLIC BEVERAGES AND TOBACCO": "Alcohol_and_tobacco",
    "05.6 - GOODS AND SERVICES FOR ROUTINE HOUSEHOLD MAINTENANCE": "Household_consumable_goods",
    "06.1 - MEDICINES AND HEALTH PRODUCTS": "Medication",
    "09.7.4 - Stationery and drawing materials (ND)": "Stationery"
}

MONTH_ABBREVIATION_TO_NUMERIC = {
    "JAN": "01",
    "FEB": "02",
    "MAR": "03",
    "APR": "04",
    "MAY": "05",
    "JUN": "06",
    "JUL": "07",
    "AUG": "08",
    "SEP": "09",
    "OCT": "10",
    "NOV": "11",
    "DEC": "12",
    "AVE":"NA",
}

DIR_INIT = [PROCESSED_OUTPUT_DIR,
PROCESSED_DATASET_REFORMATTED_DIR,]


In [3]:
for directory in DIR_INIT:
    if not os.path.exists(directory):
        os.mkdir(directory)

## Formatting Stage

### Loading CSV as a Dataframe

In [4]:
csv_data = pd.read_csv(FMCG_DATASET_FILEPATH)
csv_data

,Geolocation,Commodity Description,2018 Jan,2018 Feb,2018 Mar,2018 Apr,2018 May,2018 Jun,2018 Jul,2018 Aug,...,2024 Apr,2024 May,2024 Jun,2024 Jul,2024 Aug,2024 Sep,2024 Oct,2024 Nov,2024 Dec,2024 Ave
0,PHILIPPINES,01 - FOOD AND NON-ALCOHOLIC BEVERAGES,97.1000000000000,97.4000000000000,97.6000000000000,98.0000000000000,98.1000000000000,98.8000000000000,99.9000000000000,101.6000000000000,...,128.7000000000000,128.7000000000000,129.5000000000000,130.4000000000000,130.4000000000000,129.8000000000000,..,..,..,..
1,PHILIPPINES,02 - ALCOHOLIC BEVERAGES AND TOBACCO,92.0000000000000,96.3000000000000,98.1000000000000,99.3000000000000,99.9000000000000,100.5000000000000,101.3000000000000,101.7000000000000,...,176.5000000000000,176.8000000000000,177.0000000000000,177.0000000000000,177.2000000000000,177.5000000000000,..,..,..,..
2,PHILIPPINES,05.6 - GOODS AND SERVICES FOR ROUTINE HOUSEHOL...,98.3000000000000,98.5000000000000,99.1000000000000,99.3000000000000,99.4000000000000,99.6000000000000,100.2000000000000,100.5000000000000,...,125.2000000000000,125.5000000000000,125.6000000000000,125.8000000000000,126.0000000000000,126.2000000000000,..,..,..,..
3,PHILIPPINES,06.1 - MEDICINES AND HEALTH PRODUCTS,98.6000000000000,98.9000000000000,99.2000000000000,99.4000000000000,99.7000000000000,100.0000000000000,100.1000000000000,100.4000000000000,...,119.9000000000000,120.1000000000000,120.3000000000000,120.6000000000000,120.7000000000000,121.0000000000000,..,..,..,..
4,PHILIPPINES,09.7.4 - Stationery and drawing materials (ND),98.1000000000000,98.3000000000000,98.6000000000000,98.7000000000000,99.1000000000000,100.0000000000000,100.4000000000000,100.9000000000000,...,130.3000000000000,130.5000000000000,130.8000000000000,131.7000000000000,133.3000000000000,133.7000000000000,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,....City of Butuan (Capital),01 - FOOD AND NON-ALCOHOLIC BEVERAGES,96.8000000000000,97.3000000000000,96.5000000000000,98.6000000000000,98.1000000000000,98.4000000000000,99.0000000000000,100.6000000000000,...,132.3000000000000,131.7000000000000,131.0000000000000,131.1000000000000,131.2000000000000,131.3000000000000,..,..,..,..
591,....City of Butuan (Capital),02 - ALCOHOLIC BEVERAGES AND TOBACCO,85.0000000000000,94.3000000000000,95.6000000000000,96.6000000000000,97.5000000000000,98.2000000000000,102.2000000000000,105.6000000000000,...,174.2000000000000,174.1000000000000,174.2000000000000,174.3000000000000,174.3000000000000,173.9000000000000,..,..,..,..
592,....City of Butuan (Capital),05.6 - GOODS AND SERVICES FOR ROUTINE HOUSEHOL...,99.5000000000000,99.5000000000000,99.5000000000000,99.5000000000000,99.5000000000000,99.5000000000000,99.6000000000000,99.6000000000000,...,117.2000000000000,117.2000000000000,117.2000000000000,117.3000000000000,117.3000000000000,117.3000000000000,..,..,..,..
593,....City of Butuan (Capital),06.1 - MEDICINES AND HEALTH PRODUCTS,98.9000000000000,98.9000000000000,99.2000000000000,99.5000000000000,99.6000000000000,99.7000000000000,100.2000000000000,100.2000000000000,...,134.7000000000000,135.2000000000000,135.2000000000000,135.2000000000000,135.2000000000000,135.2000000000000,..,..,..,..


In [5]:
def remove_trailing_dots(string):
    return re.sub(r'^\.+', '', string)

def remap_commodity_descriptions(string):
    return COMMODITY_MAPPING[string]

csv_data['Geolocation'] = csv_data['Geolocation'].apply(remove_trailing_dots)
csv_data['Commodity Description'] = csv_data['Commodity Description'].apply(remap_commodity_descriptions)
csv_data = csv_data.loc[:, ~csv_data.columns.str.endswith('Ave')]
csv_data = csv_data.astype(str)
csv_data

,Geolocation,Commodity Description,2018 Jan,2018 Feb,2018 Mar,2018 Apr,2018 May,2018 Jun,2018 Jul,2018 Aug,...,2024 Mar,2024 Apr,2024 May,2024 Jun,2024 Jul,2024 Aug,2024 Sep,2024 Oct,2024 Nov,2024 Dec
0,PHILIPPINES,Food_and_drinks,97.1000000000000,97.4000000000000,97.6000000000000,98.0000000000000,98.1000000000000,98.8000000000000,99.9000000000000,101.6000000000000,...,128.9000000000000,128.7000000000000,128.7000000000000,129.5000000000000,130.4000000000000,130.4000000000000,129.8000000000000,..,..,..
1,PHILIPPINES,Alcohol_and_tobacco,92.0000000000000,96.3000000000000,98.1000000000000,99.3000000000000,99.9000000000000,100.5000000000000,101.3000000000000,101.7000000000000,...,176.3000000000000,176.5000000000000,176.8000000000000,177.0000000000000,177.0000000000000,177.2000000000000,177.5000000000000,..,..,..
2,PHILIPPINES,Household_consumable_goods,98.3000000000000,98.5000000000000,99.1000000000000,99.3000000000000,99.4000000000000,99.6000000000000,100.2000000000000,100.5000000000000,...,125.1000000000000,125.2000000000000,125.5000000000000,125.6000000000000,125.8000000000000,126.0000000000000,126.2000000000000,..,..,..
3,PHILIPPINES,Medication,98.6000000000000,98.9000000000000,99.2000000000000,99.4000000000000,99.7000000000000,100.0000000000000,100.1000000000000,100.4000000000000,...,119.8000000000000,119.9000000000000,120.1000000000000,120.3000000000000,120.6000000000000,120.7000000000000,121.0000000000000,..,..,..
4,PHILIPPINES,Stationery,98.1000000000000,98.3000000000000,98.6000000000000,98.7000000000000,99.1000000000000,100.0000000000000,100.4000000000000,100.9000000000000,...,129.9000000000000,130.3000000000000,130.5000000000000,130.8000000000000,131.7000000000000,133.3000000000000,133.7000000000000,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,City of Butuan (Capital),Food_and_drinks,96.8000000000000,97.3000000000000,96.5000000000000,98.6000000000000,98.1000000000000,98.4000000000000,99.0000000000000,100.6000000000000,...,133.7000000000000,132.3000000000000,131.7000000000000,131.0000000000000,131.1000000000000,131.2000000000000,131.3000000000000,..,..,..
591,City of Butuan (Capital),Alcohol_and_tobacco,85.0000000000000,94.3000000000000,95.6000000000000,96.6000000000000,97.5000000000000,98.2000000000000,102.2000000000000,105.6000000000000,...,174.2000000000000,174.2000000000000,174.1000000000000,174.2000000000000,174.3000000000000,174.3000000000000,173.9000000000000,..,..,..
592,City of Butuan (Capital),Household_consumable_goods,99.5000000000000,99.5000000000000,99.5000000000000,99.5000000000000,99.5000000000000,99.5000000000000,99.6000000000000,99.6000000000000,...,116.9000000000000,117.2000000000000,117.2000000000000,117.2000000000000,117.3000000000000,117.3000000000000,117.3000000000000,..,..,..
593,City of Butuan (Capital),Medication,98.9000000000000,98.9000000000000,99.2000000000000,99.5000000000000,99.6000000000000,99.7000000000000,100.2000000000000,100.2000000000000,...,134.1000000000000,134.7000000000000,135.2000000000000,135.2000000000000,135.2000000000000,135.2000000000000,135.2000000000000,..,..,..


In [6]:
transposed_csv = csv_data.transpose()
transposed_csv

,0,1,2,3,4,5,6,7,8,9,...,585,586,587,588,589,590,591,592,593,594
Geolocation,PHILIPPINES,PHILIPPINES,PHILIPPINES,PHILIPPINES,PHILIPPINES,National Capital Region (NCR),National Capital Region (NCR),National Capital Region (NCR),National Capital Region (NCR),National Capital Region (NCR),...,Dinagat Islands,Dinagat Islands,Dinagat Islands,Dinagat Islands,Dinagat Islands,City of Butuan (Capital),City of Butuan (Capital),City of Butuan (Capital),City of Butuan (Capital),City of Butuan (Capital)
Commodity Description,Food_and_drinks,Alcohol_and_tobacco,Household_consumable_goods,Medication,Stationery,Food_and_drinks,Alcohol_and_tobacco,Household_consumable_goods,Medication,Stationery,...,Food_and_drinks,Alcohol_and_tobacco,Household_consumable_goods,Medication,Stationery,Food_and_drinks,Alcohol_and_tobacco,Household_consumable_goods,Medication,Stationery
2018 Jan,97.1000000000000,92.0000000000000,98.3000000000000,98.6000000000000,98.1000000000000,97.4000000000000,91.6000000000000,98.3000000000000,98.4000000000000,98.2000000000000,...,94.9000000000000,88.0000000000000,99.8000000000000,98.3000000000000,99.8000000000000,96.8000000000000,85.0000000000000,99.5000000000000,98.9000000000000,99.9000000000000
2018 Feb,97.4000000000000,96.3000000000000,98.5000000000000,98.9000000000000,98.3000000000000,97.7000000000000,96.6000000000000,98.5000000000000,98.7000000000000,98.5000000000000,...,98.3000000000000,91.6000000000000,100.0000000000000,98.3000000000000,99.8000000000000,97.3000000000000,94.3000000000000,99.5000000000000,98.9000000000000,99.9000000000000
2018 Mar,97.6000000000000,98.1000000000000,99.1000000000000,99.2000000000000,98.6000000000000,98.0000000000000,99.1000000000000,99.3000000000000,99.4000000000000,99.6000000000000,...,98.1000000000000,97.9000000000000,100.0000000000000,99.1000000000000,99.8000000000000,96.5000000000000,95.6000000000000,99.5000000000000,99.2000000000000,99.9000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024 Aug,130.4000000000000,177.2000000000000,126.0000000000000,120.7000000000000,133.3000000000000,129.1000000000000,147.1000000000000,123.6000000000000,111.5000000000000,118.9000000000000,...,132.8000000000000,169.5000000000000,179.0000000000000,146.9000000000000,167.8000000000000,131.2000000000000,174.3000000000000,117.3000000000000,135.2000000000000,142.8000000000000
2024 Sep,129.8000000000000,177.5000000000000,126.2000000000000,121.0000000000000,133.7000000000000,129.5000000000000,147.3000000000000,123.6000000000000,111.7000000000000,119.1000000000000,...,131.8000000000000,170.5000000000000,179.7000000000000,146.4000000000000,167.8000000000000,131.3000000000000,173.9000000000000,117.3000000000000,135.2000000000000,142.8000000000000
2024 Oct,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2024 Nov,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [7]:
def convert_date(string):
    year, month = string.split()
    month_numeric = MONTH_ABBREVIATION_TO_NUMERIC[month.upper()]
    return f"{year}-{month_numeric}"

In [8]:
transposed_data = transposed_csv.copy()
transposed_data.reset_index()
transposed_data_date = transposed_data.index.copy()[2:]
transposed_data_date = transposed_data_date.map(convert_date)
data_array = transposed_data.to_numpy()
horizontal_limit = len(data_array[0])
x = 0
while x < horizontal_limit:
    location = data_array[0][x]
    temp_df = pd.DataFrame()
    temp_df["Date"] = transposed_data_date
    temp_df["Food_and_drinks"] = data_array[2:,x + 0]
    temp_df["Alcohol_and_tobacco"] = data_array[2:,x + 1]
    temp_df["Household_consumable_goods"] = data_array[2:,x + 2]
    temp_df["Medication"] = data_array[2:,x + 3]
    temp_df["Stationery"] = data_array[2:,x + 4]
    output_df = temp_df[~temp_df.apply(lambda row: row.astype(str).str.endswith('..').any(), axis=1)]
    output_df.to_csv(PROCESSED_DATASET_REFORMATTED_DIR + f"{location}.csv")
    
    x+=5